In [296]:
import numpy as np
import pandas as pd

In [297]:
findf = pd.read_csv("./Data/finalData/findf_AllSeries.csv", index_col=0)
findf.RSSD9999 = pd.to_datetime(findf.RSSD9999, format = "%Y-%m-%d")
findf = findf.sort_values(["RSSD9999", "RSSD9001"])

# Remove Bank Holding Companies
attributesOpen = pd.read_csv("./Data/MergerAdjust/CSV_ATTRIBUTES_ACTIVE.csv", low_memory=False)
attributesClosed = pd.read_csv("./Data/MergerAdjust/CSV_ATTRIBUTES_CLOSED.csv", low_memory=False)
attributesBranches = pd.read_csv("./Data/MergerAdjust/CSV_ATTRIBUTES_BRANCHES.csv", low_memory=False)
attributes = pd.concat([attributesOpen, attributesClosed, attributesBranches], axis = 0).rename({"#ID_RSSD":"RSSD9001"}, axis = 1)
bhcInd = attributes[["RSSD9001", "NM_LGL", "BHC_IND"]]

findf = pd.merge(findf, bhcInd, on = "RSSD9001", how = "left")

findf = findf[findf.BHC_IND == 0]

findf = findf.drop(["BHC_IND", "NM_LGL"], axis = 1)

In [298]:
# # Merger adjust the data

# transformations = pd.read_csv("./Data/MergerAdjust/CSV_TRANSFORMATIONS.CSV")
# transformations.rename({"#ID_RSSD_PREDECESSOR":"oldRSSD9001", "ID_RSSD_SUCCESSOR":"newRSSD9001"}, axis = 1, inplace = True)
# transformations = transformations[~transformations.TRNSFM_CD.isin([5, 7])] # Remove splits and sale of assets since they don't result in a new entity
# transformations = transformations[["oldRSSD9001", "newRSSD9001", "TRNSFM_CD", "ACCT_METHOD", "DT_TRANS"]]
# transformations["nAcquirers"] = transformations.apply(lambda x: transformations[(transformations.oldRSSD9001 == x["oldRSSD9001"]) & (transformations.DT_TRANS == x["DT_TRANS"])].shape[0], axis = 1)
# transformations = transformations.sort_values("DT_TRANS")
# transformations.DT_TRANS = pd.to_datetime(transformations.DT_TRANS, format = "%Y%m%d")

# def getUltimateCompany(oldRSSD):
#     if transformations[transformations.oldRSSD9001 == oldRSSD].shape[0] == 0:
#         return oldRSSD
#     elif transformations[transformations.oldRSSD9001 == oldRSSD].shape[0] == 1:
#         return getUltimateCompany(transformations[transformations.oldRSSD9001 == oldRSSD].newRSSD9001.values[0])
#     else:
#         return [getUltimateCompany(x) for x in transformations[transformations.oldRSSD9001 == oldRSSD].newRSSD9001.values]

# transformations.loc[:,"ultimateCompany"] = transformations.apply(lambda x: getUltimateCompany(x["oldRSSD9001"]), axis = 1)

# while transformations.ultimateCompany.apply(lambda x: type(x)).value_counts().shape[0] > 1:
#     transformations = transformations.explode("ultimateCompany")

# transformations = transformations.drop_duplicates(["oldRSSD9001", "ultimateCompany", "DT_TRANS"])
# transformations = transformations[["oldRSSD9001", "ACCT_METHOD", "TRNSFM_CD", "DT_TRANS", "nAcquirers", "ultimateCompany"]]

# transformations.to_csv("./Data/MergerAdjust/transformations.csv")

In [299]:
findf.shape

(1973172, 176)

In [300]:
transformations = pd.read_csv("./Data/MergerAdjust/transformations.csv", index_col=0)
transformations.DT_TRANS = pd.to_datetime(transformations.DT_TRANS, format = "%Y-%m-%d")

In [301]:
balancesheet = ["RCFD2170", "RCON2170", "RCFN2170", "RCFD3368", "RCON3368", "RCFN3368", "RCON2192", "RCON2948", "RCON2215", "RCON6810", "RCON0352", "RCON6648", "RCON6645", "RCON6646", "RCON2604", "RCONJ473", "RCONJ474", "RCONA223", "RCON0071", "RCON2122", "RCON0390", "RCON2146", "RCON3545", "RCON1754", "RCON1772", "RCON1350", "RCONB987", "RCONB989", "RCON8725", "RCON3450", "RCONA126", "RCONA589", "RIAD3210", "RCON3210", "RCON2385", "RCON3809", "RCON6631", "RCON6636", "RCON8693", "RCON8697", "RCON8701", "RCON8705", "RCON8709", "RCON8713", "RCON8733", "RCON8737", "RCON8741", "RCON8745", "RCON8766", "RCON8767", "RCONS582", "RCONS583", "RCONS584", "RCONS603", "RCONS604", "RCONS605", "RCON0081", 'RCONA549', 'RCONA550', 'RCONA551', 'RCONA552', 'RCONA553', 'RCONA554', 'RCONA555', 'RCONA556', 'RCONA557', 'RCONA558', 'RCONA559', 'RCONA560', 'RCONA561', 'RCONA562', 'RCONA564', 'RCONA565', 'RCONA566', 'RCONA567', 'RCONA568', 'RCONA569', 'RCONA570', 'RCONA571', 'RCONA572', 'RCONA573', 'RCONA574', 'RCONA575', "RCONA579", "RCONA580", "RCONA581", "RCONA582", "RCONA584", "RCONA585", "RCONA586", "RCONA587", "RCONHK07", "RCONHK08", "RCONHK09", "RCONHK10", "RCONHK12", "RCONHK13", "RCONHK14", "RCONHK15", "RCONF055", "RCONF056", "RCONF057", "RCONF058", "RCONF060", "RCONF061", "RCONF062", "RCONF063"]
incomestatement = ["RIAD4340", "RIADC914", "RIAD4341", "RIAD4074", "RIADC899", "RIADC900", "RIADB525", "RIAD4842", "RIAD4079", "RIAD4093", "RIADC902", "RIADC903", "RIADC904", "RIADC905", "RIADC907", "RIAD4097", "RIAD4239", "RIAD4230", "RIADJJ33", "RIADKW02", "RIADC901", "RIAD4235", "RIAD4107", "RIAD4073", "RIAD4115", "RIAD4020", "RIAD4010", "RIAD4011", "RIAD4012", "RIAD4013", "RIAD4060", "RIAD4069", "RIAD4075", "RIAD4077", "RIAD4091", "RIAD4092", "RIAD4101", "RIAD4126", "RIAD4170", "RIAD4185", "RIAD4200", "RIAD4218", "RIAD4435", "RIAD4436", "RIAD4507", "RIAD4508", "RIAD4513", "RIAD4518", "RIAD4619", "RIAD5466", "RIAD8622", "RIAD8757", "RIADA315", "RIADA517", "RIADA518", "RIADB488", "RIADB489", "RIADB497", "RIADGW44", "RIADHK03", "RIADHK04", "RIADHT73", "RIADHT74", "RIADHT85"]

In [302]:
findfMerger = pd.merge(findf, transformations, how = "left", left_on = "RSSD9001", right_on = "oldRSSD9001")

In [303]:
relQuarterStart = pd.to_datetime(findfMerger.DT_TRANS.dt.year.map("{:.0f}".format) + "-" + (findfMerger.DT_TRANS.dt.quarter*3-2).map("{:.0f}".format) + "-"  + "1", format = "%Y-%m-%d", errors = "coerce")
relQuarterEnd = pd.to_datetime(findfMerger.DT_TRANS.dt.year.map("{:.0f}".format) + "-" + (findfMerger.DT_TRANS.dt.quarter*3).map("{:.0f}".format) + "-"  + (30 + (findfMerger.DT_TRANS.dt.quarter.isin([1,4]))).map("{:.0f}".format), format = "%Y-%m-%d", errors = "coerce")

In [304]:
findfMerger.loc[:,"discount"] = findfMerger.ACCT_METHOD.fillna(1).replace([2,0], np.nan).fillna((findfMerger.RSSD9999 - findfMerger.DT_TRANS).dt.days/(relQuarterEnd - relQuarterStart).dt.days)
findfMerger.loc[findfMerger.discount < 0, "discount"] = 1
findfMerger.loc[findfMerger.discount > 1, "discount"] = 1

In [305]:
findfMerger.loc[:,balancesheet] = findfMerger[balancesheet] / findfMerger.nAcquirers.fillna(1).astype(int).values.reshape(-1,1)
findfMerger.loc[:,incomestatement] = findfMerger[incomestatement] / findfMerger.nAcquirers.fillna(1).astype(int).values.reshape(-1,1) * findfMerger.discount.fillna(1).values.reshape(-1,1)
findfMerger.loc[:,"RSSD9001"] = findfMerger.loc[:, "ultimateCompany"].fillna(findfMerger.loc[:, "RSSD9001"])

In [306]:
mergeCompanyQuarters = [(a,b) for a,b in zip(findfMerger.RSSD9001[~findfMerger.DT_TRANS.isna()].values, findfMerger.DT_TRANS.dt.to_period("Q")[~findfMerger.DT_TRANS.isna()].values)]
mergeCompanyQuarters = list(set(mergeCompanyQuarters))

In [307]:
findfMerger = findfMerger.groupby(["RSSD9001", "RSSD9999"]).sum(balancesheet + incomestatement)
findfMerger.shape

(831720, 180)

In [308]:
findfMerger["dateQ"] = findfMerger.index.get_level_values(1).to_period("Q")
findfMerger = findfMerger.reset_index().set_index(["RSSD9001", "dateQ"])

In [309]:
mergeCompanyQuarters = pd.DataFrame(mergeCompanyQuarters)
mergeCompanyQuarters = mergeCompanyQuarters[pd.MultiIndex.from_frame(mergeCompanyQuarters).isin(findfMerger.index)].values
mergeCompanyQuarters = list(map(tuple, mergeCompanyQuarters))

In [310]:
findfMerger["MADummy"] = 0
findfMerger.loc[mergeCompanyQuarters, "MADummy"] = 1
findfMerger = findfMerger.reset_index().set_index(["RSSD9001", "RSSD9999"])

In [311]:
findfMerger = findfMerger[balancesheet + incomestatement + ["MADummy"]]
findfMerger.MADummy.value_counts()

MADummy
0    822632
1      9088
Name: count, dtype: int64

In [312]:
findfMerger.to_csv("./Data/finalData/findf_AllSeries_MergerAdjusted.csv")